<a href="https://colab.research.google.com/github/saulrcohen/Projects/blob/master/HalachaProject/HalachaEmails.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Reading data and removing non-questions (Slowly)

In [ ]:
#Import important packages
import pandas as pd
import numpy as np
import nltk
import io
import re
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
lemmatizer = WordNetLemmatizer()
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
stop_words = set(stopwords.words('english'))



In [ ]:
my_file = open("listfile.txt", "r")
content = my_file.read()
content = content.split('\n')

In [ ]:
len(content)
str_list = list(filter(None, content))
len(str_list)

12086

In [ ]:
content = [s for s in content if 'subscription preferences' not in s]
content = [s for s in content if 'unsubscribe from' not in s]
content = [s for s in content if 'mailchimp' not in s]
content = [s for s in content if 'MailChimp' not in s]
content = [s for s in content if 'Mailchimp' not in s]
content = [s for s in content if 'mailing list' not in s]
content = [s for s in content if '========' not in s]
content = [s for s in content if 'Hamikubalim 7' not in s]
content = [s for s in content if 'Follow me on ' not in s]
content = [s for s in content if '@RavJesseHorn' not in s]
content = [s for s in content if 'This email was sent to' not in s]
content = [s for s in content if 'Designing Your Email' not in s]
content = [s for s in content if 'Creating an elegant email is simple' not in s]
content = [s for s in content if '-----------' not in s]
content = [s for s in content if 'Sign up for weekly Halachic Questions' not in s]
content = [s for s in content if 'All rights reserved' not in s]
content = [s for s in content if 'Our mailing address is: Torah' not in s]
content = [s for s in content if 'All rights reserved' not in s]


In [ ]:
content

### Find retractions

In [ ]:
#Keep track of changes
for index in range(1, len(content)):
  #Find any time it's a new Halacha
  if content[index-1].startswith('Halacha'):
    #Means that it doesn't start with a number
    if content[index][0].isdigit() is False:
      #There's a solid chance it's a Retraction since it's not the first question
      if content[index].startswith('Halacha') is False:
        content[index] = 'Retraction in -> ' + content[index-1] + " " + content[index]


'\n#Keep track of changes\nfor index in range(len(content)):\n  #This is because I\'m bad at computer science sometimes\n  if index > 0:\n    #Find any time it\'s a new Halacha\n    if content[index-1].startswith(\'Halacha\'):\n      #Means that it doesn\'t start with a number\n      if content[index][0].isdigit() is False:\n        #There\'s a solid chance it\'s a Retraction since it\'s not the first question \n        #Checked these manually, but still going to have to be checked \n        if content[index].startswith(\'Halacha\') is False:\n          content[index] = \'Retraction in -> \' + content[index-1] + " " + content[index]\n          '

###

## Found the retractions and dealt with them, edit out whitepaces, and put questions and answers together

In [ ]:
content =  [x for x in content if not x.startswith("Halacha")]
content = [x for x in content if x]
content = [x.strip() for x in content]
content = [x.replace(':)', '') for x in content]

In [ ]:
content[0]

'1. Which way should one face at the Kotel when davening, towards the wall or towards the left facing the Makom HaMikdash?'

In [ ]:

#Concat all questions and answers until I hit another question
for question in range(len(content)):
  # This string is an answer
  if content[question][0].isdigit() == False:
    #Concat the answer with the previous question
    content[question-1] += ' ' + content[question]
    del content[question]
    #Check how many lines the answer is
    #Concat all of those back to the original question
    #Get rid of the answer
    #Start at the same answer, and concat to the previous one
  if content[question][0].isdigit() == False:
    #Check all the subsequent answer strings
    content[question - 1] = content[question - 1] + " " + content[question + i]
    del content[question + i]

In [ ]:
content = [x for x in content if x]
len(content)

3009

In [ ]:
'''
Here we have to find multi line questions.
So we check how it starts and if it's not a number but it's a letter, add it to the question at hand.
'''

for questionNumber in range(len(content) - 3):
  if content[questionNumber][0].isdigit() == False and content[questionNumber][0].isalnum():
    content[questionNumber-1] += " " + content[questionNumber]
    content[questionNumber] = ""
    content = [x for x in content if x]

## Create the NLP model to group the questions



In [ ]:
df = pd.DataFrame(content)
df.head()

,0
0,1. Which way should one face at the Kotel when...
1,"2. When playing ""Settlers of Katan"" on Shabbo..."
2,3. Should I wear woolen Tzitzis? There is a Ma...
3,1. Which way should one face at the Kotel when...
4,"2. When playing ""Settlers of Katan"" on Shabbo..."


In [ ]:
vals = df[0].unique().astype('U')

In [ ]:
def preprocess_text(text):
    tokens = word_tokenize(text.lower())
    tokens = [lemmatizer.lemmatize(token) for token in tokens if token.isalpha()]
    tokens = [token for token in tokens if token not in stop_words]
    return " ".join(tokens)

In [ ]:
# Preprocess questions
preprocessed_questions = [preprocess_text(question) for question in questions]

# Vectorize the questions
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(preprocessed_questions)

# Perform clustering
num_clusters = 10  # Adjust the number of clusters as desired
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
kmeans.fit(X)


In [ ]:
categorized_questions = {}
for i, question in enumerate(questions):
    category = cluster_labels[i]
    categorized_questions.setdefault(category, []).append(question)

# Print categorized questions
for category, questions in categorized_questions.items():
    print("Category", category+1)
    for question in questions:
        print("- " + question)
    print()